In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import glob  

current_date = datetime.now()
formatted_date = current_date.strftime("%d-%b-%Y")


def getCsvFileName():
    # Get the current date
    current_date = datetime.now()

    # Format the date as 'dd-MMM-yyyy'
    formatted_date = current_date.strftime("%d-%b-%Y")

    # Create the new filename with the current date
    filename = f"CF-Event-equities-{formatted_date}.csv"
    return filename

def getBulkCsvFileName():
    today = datetime.date.today()  # Add parentheses here  
    start_date = today - datetime.timedelta(days=7)  
    end_date = today  

    file_name = f"Bulk-Deals-{start_date.strftime('%d-%m-%Y')}-to-{end_date.strftime('%d-%m-%Y')}.csv"  
    print(file_name)
    return file_name

def download_pdf(url ,buttonName):
    # Initialize the WebDriver using WebDriver Manager
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
    # Open the NSE India corporate filings event calendar page
    driver.get(url)
    
    try:
        # Wait for the download button to be clickable
        wait = WebDriverWait(driver, 20)
        download_button = wait.until(EC.element_to_be_clickable((By.ID,buttonName)))

        # Click the download button
        download_button.click()
        
        # Wait for the file to download
        time.sleep(5)  # Adjust sleep time as needed based on download speed

        # Assuming the file is downloaded to the default download directory
        download_dir = os.path.expanduser('~/Downloads')
        downloaded_file_path = os.path.join(download_dir, 'Corporate Filings Event Calendar.csv')

        # Check if the file was downloaded successfully
        # if os.path.exists(downloaded_file_path):
        #     print(f"File downloaded successfully: {downloaded_file_path}")
        # else:
        #     print("File download failed.")
    finally:
        # Close the WebDriver
        driver.quit()

filename=getCsvFileName()
download_dir = os.path.expanduser('~/Downloads')
downloaded_file_path = os.path.join(download_dir, filename)

print(filename)
#Download earning dates
for i in range(1, 10):  
    if os.path.exists(downloaded_file_path):
        print(f"File downloaded successfully: {downloaded_file_path}")
        break     
    download_pdf('https://www.nseindia.com/companies-listing/corporate-filings-event-calendar',"CFeventCalendar-download")

# Load the CSV into a DataFrame
earnings_df = pd.read_csv(downloaded_file_path)
earnings_df.columns = earnings_df.columns.str.strip().str.replace('\n', '', regex=False)
earnings_df = earnings_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

print(earnings_df.head)

#downloaded_file_path = os.path.join(download_dir, getBulkCsvFileName())
downloaded_file_path = os.path.join(download_dir,"Bulk-Deals-15-07-2024-to-22-07-2024.csv")

print(filename)
# Download earning dates
for i in range(1, 10):  
    if os.path.exists(downloaded_file_path):
        print(f"File downloaded successfully: {downloaded_file_path}")
        break    
    download_pdf("https://www.nseindia.com/report-detail/display-bulk-and-block-deals",'HistBulkBlockDeals-download')


# Load the CSV into a DataFrame
bulk_deals_df = pd.read_csv(downloaded_file_path)
bulk_deals_df.columns = bulk_deals_df.columns.str.strip().str.replace('\n', '', regex=False)
bulk_deals_df = bulk_deals_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

print(bulk_deals_df)


In [ ]:
# Rename columns to match
bulk_deals_df.rename(columns={'Symbol':'SYMBOL'}, inplace=True)
# Rename columns to match
bulk_deals_df.rename(columns={'Buy / Sell':'BUY_SELL'}, inplace=True)


In [ ]:
# # Add a helper column for sorting and grouping
# bulk_deals_df['Helper'] = bulk_deals_df.apply(lambda x: (x['Date'], x['SYMBOL'], x['Client Name'], x['Quantity Traded']), axis=1)

# # Group by helper column and filter out the groups with both buy and sell
# grouped = bulk_deals_df.groupby('Helper').filter(lambda x: not ('BUY' in x['BUY_SELL'].values and 'SELL' in x['BUY_SELL'].values))

# # Drop the helper column
# bulk_deals_df = grouped.drop(columns=['Helper'])

# bulk_deals_df.to_csv("remove_dep.csv")
# # Print the resulting DataFrame
# print(bulk_deals_df)

In [43]:
# Merge DataFrames on 'SYMBOL'
merged_df = pd.merge(earnings_df, bulk_deals_df, on='SYMBOL', how='inner')
merged_df.drop(columns=['DETAILS','Security Name','Remarks','Date'], inplace=True)
merged_df['Quantity Traded'] = merged_df['Quantity Traded'].str.replace(',', '').astype(int)
merged_df['Quantity Traded'] = merged_df.apply(
    lambda row: -row['Quantity Traded'] if row['BUY_SELL'] == 'SELL' else row['Quantity Traded'],
    axis=1
)
merged_df.to_csv('results/matching_bulk_stocks.csv', index=False)


# client level orders

In [55]:
# Group by 'SYMBOL' and 'Client Name' and aggregate the values
result_df = merged_df.groupby(['SYMBOL', 'Client Name']).agg({
    'Quantity Traded': 'sum',
    'Trade Price / Wght. Avg. Price': lambda x: list(x)
}).reset_index()

client_level_group = result_df[result_df['Quantity Traded'] != 0]

client_level_group.to_csv("results/final_client_level.csv")

print(client_level_group)

       SYMBOL                                        Client Name  \
2   CYBERTECH                               HRTI PRIVATE LIMITED   
3    GODREJCP                                  R K N ENTERPRISES   
7        MTNL                               AAKRAYA RESEARCH LLP   
8        MTNL                              BONANZA PORTFOLIO LTD   
10       MTNL            DIPAN MEHTA COMMODITIES PRIVATE LIMITED   
11       MTNL                        F3 ADVISORS PRIVATE LIMITED   
13       MTNL                               HRTI PRIVATE LIMITED   
14       MTNL                             JAINAM BROKING LIMITED   
15       MTNL                    JSI INVESTMENTS PRIVATE LIMITED   
16       MTNL                               KAMLESH BABALAL SHAH   
17       MTNL             MANSI SHARE AND STOCK ADVISORS PVT LTD   
19       MTNL  MULTIPLIER SHARE & STOCK ADVISORS PRIVATE LIMITED   
21       MTNL                 NAMAN SECURITIES & FINANCE PVT LTD   
23       MTNL                                  Q

In [56]:
# Group by 'SYMBOL' and 'Client Name' and aggregate the values
stock_level_group = client_level_group.groupby(['SYMBOL']).agg({
    'Quantity Traded': 'sum',
        'Trade Price / Wght. Avg. Price': lambda x: list(x),

    'Client Name': lambda x: list(x),
        ' Quantity Traded': lambda x: list(x)

   
}).reset_index()


stock_level_group.to_csv("results/final_stock_level.csv")

print(stock_level_group)

      SYMBOL  Quantity Traded  \
0  CYBERTECH             2875   
1   GODREJCP        -11392858   
2       MTNL         -4865508   
3   ONMOBILE            45826   
4       TTML          2049376   

                      Trade Price / Wght. Avg. Price  
0                                 [[206.00, 206.42]]  
1                                       [[1,443.90]]  
2  [[53.35, 53.40, 60.53, 60.59, 68.96, 69.04], [...  
3                   [[85.36, 85.39], [85.66, 85.86]]  
4  [[105.46, 105.51], [90.92, 90.17, 105.34, 106....  


In [36]:
# Step 2: Remove rows where the same "Client Name" has BUY and SELL with the same "Quantity Traded"
# Creating a key based on Client Name and Quantity Traded
merged_df['key'] = merged_df['Client Name'] + '-' + merged_df['Quantity Traded'].astype(str)

# Grouping by the key and filtering out groups that have both BUY and SELL with the same quantity
filtered_df = merged_df.groupby('key').filter(lambda x: not (x['BUY_SELL'].nunique() > 1 and len(x) > 1))

# Dropping the temporary key column
filtered_df = filtered_df.drop(columns=['key'])

# Step 3: Group by SYMBOL, COMPANY, and BUY_SELL, and aggregate the results
result_df = filtered_df.groupby(['SYMBOL', 'BUY_SELL']).agg(
    clients=('Client Name', lambda x: ', '.join(x.unique())),
    total_quantity_traded=('Quantity Traded', 'sum')
).reset_index()

# Renaming columns as needed
result_df.rename(columns={
    'SYMBOL': 'SYMBOL',
    'Security Name': 'COMPANY',
    'Buy / Sell': 'BUY_SELL',
    'clients': 'Client Names',
    'total_quantity_traded': 'Total Quantity Traded'
}, inplace=True)

# Save the result to a CSV file
result_df.to_csv("results/filtered.csv", index=False)

result_df.head(), output_path

(      SYMBOL BUY_SELL                                       Client Names  \
 0  CYBERTECH      BUY                               HRTI PRIVATE LIMITED   
 1  CYBERTECH     SELL                               HRTI PRIVATE LIMITED   
 2   GODREJCP     SELL                                  R K N ENTERPRISES   
 3       MTNL      BUY  HRTI PRIVATE LIMITED, KAMLESH BABALAL SHAH, QE...   
 4       MTNL     SELL  HRTI PRIVATE LIMITED, KAMLESH BABALAL SHAH, QE...   
 
                                Total Quantity Traded  
 0                                           1,69,205  
 1                                           1,66,330  
 2                                        1,13,92,858  
 3  71,65,2837,98,08538,11,41854,67,95836,62,45418...  
 4  72,87,24032,28,08538,94,84358,60,47638,38,5481...  ,
 '/mnt/data/processed_bulk_stocks.csv')

In [ ]:

# Group by 'SYMBOL' and 'Buy / Sell' and aggregate
result_df = merged_df.groupby(['SYMBOL', 'BUY_SELL']).agg(
    frequency=('Client Name', 'size'),
    clients=('Client Name', lambda x: list(x.unique()))
).reset_index()



Index(['SYMBOL', 'BUY_SELL', 'frequency', 'clients'], dtype='object')

In [ ]:
# Rename columns correctly
result_df.columns = ['SYMBOL', 'BUY_SELL', 'frequency', 'clients']

# Print the result DataFrame
print(result_df)

# Save to CSV
result_df.to_csv('final_result.csv', index=False)